# Option Pricing

#### By: Julián Durán
#### LinkedIn: www.linkedin.com/in/juliangodu

## Montecarlo Simulation

La simulación de Montecarlo, en este caso, se usa para valorar opciones call & put  
de tipo Europeo (Ejecución únicamente hasta el final del contrato). Para utilizar correctamente  
el modelo de pricing, introduzca el precio Spot y Strike del subyacente, el interés, la volatilidad realizada,  
el plazo y la base del contrato, y el numero de sendas a considerar.

In [3]:
from math import log, exp, sqrt
import numpy as np
from numpy import mean
from scipy.stats import norm
np.set_printoptions(precision=4, suppress=True)

In [4]:
# Datos de ejemplo
n_sendas = 1000
spot = 4000
strike = 4050
interes = 5.50
volatilidad = 19.00
plazo = 120
base = 360

In [5]:
def montecarlo_option(spot, strike, n_sendas, interes, volatilidad, plazo, base, option):
    delta_time = plazo/base
    interes = interes/100
    volatilidad = volatilidad/100
    discount_fact = exp(-interes*delta_time)
    sendas_matrix = np.zeros((n_sendas, 3))
    sendas_matrix[:,0] = np.arange(1, n_sendas + 1)
    for i in range(0, n_sendas):
        sendas_matrix[i,1] = spot*exp(((interes-(0.5*(volatilidad**2)))*delta_time)+(volatilidad*sqrt(delta_time)*norm.ppf(np.random.rand())))
        if option == 'call':
            sendas_matrix[i,2] = max(sendas_matrix[i,1] - strike, 0)
        else:
            sendas_matrix[i,2] = max(strike - sendas_matrix[i,1], 0)
    prima_opcion = mean(sendas_matrix[:,2])*discount_fact
    print(f'La prima de tu opcion {option} europea  es: {round(prima_opcion, 4)}')
    return(prima_opcion)

In [6]:
montecarlo_option(spot, strike, n_sendas, interes, volatilidad, plazo, base, 'put')

La prima de tu opcion put europea  es: 161.5832


161.58318426930305

## Black Scholes 

El modelo Black Scholes, al funcionar como formula cerrada, solo puede valorar opciones  
 call & put de tipo Europeo (Ejecución únicamente hasta el final del contrato).  
  Para utilizar correctamente el modelo de pricing, introduzca el precio Spot y Strike del subyacente,  
   la volatilidad realizada, el plazo y la base del contrato, y las tasas libre de riesgo a considerar.

$$Call_{d_1} = \frac{Ln(S_0/X) + ((r + \frac{\sigma^2}{2})\cdot \Delta T)}{\sigma\cdot \sqrt{\Delta T}}$$
$$Call_{d_2} = \frac{Ln(S_0/X) + ((r - \frac{\sigma^2}{2})\cdot \Delta T)}{\sigma\cdot \sqrt{\Delta T}}$$
$$Put_{d_1} = -(\frac{Ln(S_0/X) + ((r + \frac{\sigma^2}{2})\cdot \Delta T)}{\sigma\cdot \sqrt{\Delta T}})$$
$$Put_{d_2} = -(\frac{Ln(S_0/X) + ((r - \frac{\sigma^2}{2})\cdot \Delta T)}{\sigma\cdot \sqrt{\Delta T}})$$
$$N(d_1),\;N(d_2) \sim Z(0,1)$$
$$Prima_{call} = e^{-r\cdot \Delta T} \cdot \{ S_0 \cdot e^{r\cdot \Delta T} \cdot N(d_1) - X\cdot N(d_2)    \}  $$
$$Prima_{put} = - (e^{-r\cdot \Delta T} \cdot \{ S_0 \cdot e^{r\cdot \Delta T} \cdot N(d_1) - X\cdot N(d_2)    \})  $$

In [7]:
def blackscholes_pricing(spot, strike, dias, base, volatilidad, rf_base, rf_externa, option):
    #Parámetros
    ln_rf_base = log((1 + (rf_base/100)))
    ln_rf_externa = log((1 + (rf_externa/100)))
    r = ln_rf_base - ln_rf_externa
    delta_time = dias/base
    
    #Black Scholes
    if option == 'call':
        d1 = (log(spot/strike) + ((r + (((volatilidad/100)**2/2)))*delta_time))/((volatilidad/100)*sqrt(delta_time))
        d2 = (log(spot/strike) + ((r - (((volatilidad/100)**2/2)))*delta_time))/((volatilidad/100)*sqrt(delta_time))
        N_d1 = norm(loc = 0, scale = 1).cdf(d1)
        N_d2 = norm(loc = 0, scale = 1).cdf(d2)
        option_prime = exp(-ln_rf_base*delta_time) * (spot*exp((r*delta_time))*N_d1 - strike*N_d2)
    if option == 'put':
        d1 = (log(spot/strike) + ((r + (((volatilidad/100)**2/2)))*delta_time))/((volatilidad/100)*sqrt(delta_time))*-1
        d2 = (log(spot/strike) + ((r - (((volatilidad/100)**2/2)))*delta_time))/((volatilidad/100)*sqrt(delta_time))*-1
        N_d1 = norm(loc = 0, scale = 1).cdf(d1)
        N_d2 = norm(loc = 0, scale = 1).cdf(d2)
        option_prime = (exp(-ln_rf_base*delta_time) * (spot*exp((r*delta_time))*N_d1 - strike*N_d2))*-1
    print(f'La prima de tu opción europea es: {round(option_prime,4)}')
    return(option_prime)

In [8]:
blackscholes_pricing(spot = 4000, strike = 4400, dias = 90,
                    base = 360, volatilidad = 17, rf_base = 13.25,
                    rf_externa = 5, option = 'call')

La prima de tu opción europea es: 35.1017


35.101709499194804